In [1]:
#Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [2]:
#Enable inline plots
%matplotlib inline

In [3]:
#Import the streamflow data CSV and format columns
df=pd.read_csv('GageData.csv',dtype={'site_no':'str'},parse_dates=['datetime'])

In [4]:
#Add year, month, and water_year columns
df['year'] = df['datetime'].map(lambda x: x.year)
df['month'] = df['datetime'].map(lambda x: x.month)
df['water_year'] = df['datetime'].apply(lambda x: x.year if x.month >= 10 else x.year - 1)

In [5]:
#Compute flow in cms
df['MeanFlow_cms'] = df['MeanFlow_cfs'] * 0.028316847 

In [6]:
#Set the index to full date
df.index = df.datetime

In [7]:
#Create data slices for pre- and post-Falls Lake
dfPre = df[:'1980-01-01']
dfPost = df['1983-12-31':]

### Compute max annual flow
Compute max annual flow from our daily flow data

In [8]:
df.sort_values(by='MeanFlow_cms',ascending=False).head()

,agency_cd,site_no,datetime,MeanFlow_cfs,Confidence,year,month,water_year,MeanFlow_cms
datetime,,,,,,,,,
1945-09-19,USGS,02087500,1945-09-19,22500.0,A,1945,9,1944,637.129057
1945-09-18,USGS,02087500,1945-09-18,19800.0,A,1945,9,1944,560.673571
1999-09-17,USGS,02087500,1999-09-17,19700.0,A,1999,9,1998,557.841886
1945-09-20,USGS,02087500,1945-09-20,19600.0,A,1945,9,1944,555.010201
1996-09-07,USGS,02087500,1996-09-07,18900.0,A,1996,9,1995,535.188408


In [9]:
#Group data on water_year
byYear = df.groupby('water_year')
dfMaxAnnual = byYear['MeanFlow_cms'].max()
dfMaxAnnual.head()

In [10]:
#Covert to a dataframe
dfMA = pd.DataFrame(dfMaxAnnual)
dfMA.head()

In [11]:
### Compute rankings
dfMA['rank'] = dfMA.rank(ascending=False)
dfMA.sort_values(by='MeanFlow_cms',ascending=False,inplace=True)
dfMA.head()

In [12]:
# Compute Return intervale
countRecs = dfMA['rank'].max()
dfMA['RI'] = (countRecs + 1) / dfMA['rank']

In [13]:
# Compute probability of recurrence (POR)
dfMA['Pe'] = 1 / dfMA['RI']
#dfMA.head()

In [14]:
#Set variables to plot
x = dfMA['RI']
y = dfMA['MeanFlow_cms']

logx = np.log(x)

### Compute the regression using NumPy functions

In [15]:
# Compute the slope and intercept using the polyfit
regSlope, regIntercept = np.polyfit(np.log(x), y, 1)
print(regSlope, regIntercept)

118.000954287 128.192588643


In [16]:
#Convert the regression components into a printable formula
regText = "y = {0:.1f}x + {1:.1f}".format(slope,intercept)
print(regText)

NameError: name 'slope' is not defined

In [ ]:
# Compute p
p = np.poly1d(regSlope, regIntercept)
print(p.r)

In [ ]:
# Apply the regression to 100, 500, and 1000 year return intervals (x values)

#Create a series of the return intervals to compute
sFloodRI = pd.Series([100,500,1000],name='FloodRI')

#Compute the 30-year probability of exceedence for each return interval
sFloodPe30 = 1-(1-(1/sFloodRI))**30

#Compute the peak flow associated with those Pe values
sFloodPeak_cms = regIntercept * sFloodPe30 + regSlope

#Combine into a single dataframe
dfFlood = pd.concat([sFloodRI,sFloodPe30,sFloodPeak_cms],axis=1)
dfFlood.columns = ['RI','Pe','PeakFlow_cms']
dfFlood.head()

### Plotting
https://seaborn.pydata.org/tutorial/regression.html
https://seaborn.pydata.org/generated/seaborn.regplot.html#seaborn.regplot

In [ ]:
#Set the style to use Seaborn
sns.set(color_codes=True,font_scale=2)

In [ ]:
#Create the canvas (fig) and axes (ax) objects, setting the figure size
fig, ax = plt.subplots(figsize=(20,6))

#Set axis properties
ax.set(xlim=(1,1200),
       ylim=(0,1200),
       xscale ="log",
       label=regText
      )

#Plot the data
ax = sns.regplot(x='RI',
                 y='MeanFlow_cms',
                 data=dfMA,
                 logx=True,
                 fit_reg=True,
                 ci=None)

#Set the axis labels.
ax.set(xlabel='Return Interval (years)', 
       ylabel='Peak discharge(cms)',
       title=regText
      )

#Compute Peak discharges from 100, 500, and 1000 year intervals
for RI in (100,500,1000):
    #Compute peak discharge from RI
    peakFlow = (np.log(RI) * regSlope) + regIntercept
    #Add to the plot
    ax.plot(RI,peakFlow,'rs',markersize=8) #'rs' = red square markers

## Plot pre vs post Falls Lake 1980 data

In [ ]:
#Set the TimeSet variable to pre or post, using index (i.e. year) values 
dfMA.loc[dfMA.index < 1980,'TimeSet'] = 'PreFallsLake'
dfMA.loc[dfMA.index >= 1984,'TimeSet'] = 'PostFallsLake'

In [ ]:
#Plot multiple frames using Seaborn's FacetGrids

#Create the Facets based on values in the TimeSet field (Set above)
g = sns.FacetGrid(dfMA,col='TimeSet',size=10,legend_out=True)

#Add the regression plot to each facet
g.map(sns.regplot,'RI','MeanFlow_cms',
      data=dfMA,
      logx=True,
      fit_reg=True,
      ci=None)

#Set the axis values
for ax in g.axes.flat:
    ax.set(xlim=(0,700),  
           ylim=(0,700),
           xscale ="log"
          )

In [ ]:
#Create the canvas (fig) and axes (ax) objects, setting the figure size
fig, ax = plt.subplots(figsize=(20,16))

#Set axis properties
ax.set(xlim=(1,1200),
       ylim=(0,1200),
       xscale ="log"
      )

#Plot the data
ax = sns.regplot(x='RI',
                 y='MeanFlow_cms',
                 data=dfMA.loc[dfMA.index < 1980],
                 logx=True,
                 fit_reg=True,
                 ci=None,
                 label='Pre'
                )

#Plot the data
ax = sns.regplot(x='RI',
                 y='MeanFlow_cms',
                 data=dfMA.loc[dfMA.index >= 1984],
                 logx=True,
                 fit_reg=True,
                 ci=None,
                 label='Post'
                )

#Set the axis labels.
ax.set(xlabel='Return Interval (years)', 
       ylabel='Peak discharge(cms)'
      )

ax.legend()

#Compute Peak discharges from 100, 500, and 1000 year intervals
for RI in (100,500,1000):
    #Compute peak discharge from RI
    peakFlow = (np.log(RI) * regSlope) + regIntercept
    #Add to the plot
    ax.plot(RI,peakFlow,'rs',markersize=8) #'rs' = red square markers

In [ ]:
#Filter the data
dfPre = df[:'1979-12-31']

#Sort the records
dfPre.sort_values(by='MeanFlow_cms',ascending=False)

#Group data on water_year
byYear = df.groupby('water_year')

#Compute a series of max annual flow from the grouped data
sPeakFlow = byYear['MeanFlow_cms'].max()
sPeakFlow.name = "PeakFlow_cms"

#Compute rankings
sRank = sPeakFlow.rank(ascending=False)
sRank.name = "rank"

#Combine peakflow and rank series into a dataframe
dfPreMA = pd.concat([sPeakFlow,sRank],axis='columns')

#Count the number of years 
countRecs = dfPreMA['rank'].max()

#Compute Return Interval (RI)
dfPreMA['RI'] = (countRecs + 1) / dfPreMA['rank']

#Compute Probability of exceedance (Pe)
dfPreMA['Pe'] = 1 / dfPreMA['RI']